This script exists to couple powergenome (which sweeps location, year, and grid demand scenario to create different template folders)
with the batch csv caserunner (which sweeps carbon constraint, WEC cost, and WEC power limit to create a different replacement.csv for each location).
Notice that location is swept in both powergenome and caserunner, so this script also covers that interaction.

In [3]:
import subprocess
from pathlib import Path
import os
from juliacall import Main as jl
import subprocess
import os
import sys
import copy
%load_ext autoreload
%autoreload 2
%aimport powergenome

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
from pathlib import Path

import pandas as pd
from powergenome.generators import GeneratorClusters

from powergenome.util import (
    build_scenario_settings,
    init_pudl_connection,
    load_settings,
    check_settings
)
from powergenome.external_data import (
    make_generator_variability,
)

pd.options.display.max_columns = 200

cwd = Path.cwd()

settings_path = (
    cwd / "data_east" / "settings"
)
print(f"Using settings file: {settings_path}")

Using settings file: /home/becca/Documents/git/WEC-DECIDER/modules/CEM/data_east/settings


Make generator variability for wave as a new generator
Following [this powergenome notebook](https://github.com/PowerGenome/PowerGenome/blob/master/notebooks/Existing%20and%20new%20generators.ipynb)

Parameters needed in settings file for new resources:
- model_regions
- region_aggregations
- model_year
- target_usd_year
- atb_usd_year
- startup_fuel_use
- startup_vom_costs_mw
- startup_vom_costs_usd_year
- startup_costs_type
- startup_costs_per_cold_start_mw
- startup_costs_per_cold_start_usd_year
- existing_startup_costs_tech_map
- new_build_startup_costs
- atb_cost_case
- atb_financial_case
- atb_cap_recovery_years
- atb_new_gen
- renewables_clusters
- cost_multiplier_region_map
- cost_multiplier_technology_map

For non-ATB generator (wave), need settings to contain "additional_technologies_fn" and "additional_new_gen".
"additional_technologies_fn" should contain the follwing columns:
```
cols = [
        "technology",
        "tech_detail",
        "cost_case",
        "capex_mw",
        "capex_mwh",
        "fixed_o_m_mw",
        "fixed_o_m_mwh",
        "variable_o_m_mwh",
        "wacc_real",
        "heat_rate",
        "Cap_Size",
        "dollar_year",
    ]
```
and "additional_new_gen" should contain the names of "technology" entries you want to use.

Plan: make a single additional techs csv with all the wave combos I want. Then modify "additional_new_gen" to select which one I want, using settings management scenarios.

Confusion: where do I define the profile for the additional techs?
- gc.create_new_generators() in generators.py calls atb_new_generators in nrelatb.py
- That calls parallel_region_renewables which calls add_renewables_clusters, all in nrelatb.py
- That line is `cluster_builder = build_resource_clusters( 
            settings.get("RESOURCE_GROUPS"), settings.get("RESOURCE_GROUP_PROFILES")` if no ClusterBuilder was passed into atb_new_generators, or it uses the clusterbuilder that was passed. If it was passed, it is created through the same code but in the constructor of generators.py.
- So ultimately what's called is `cluster_builder = ClusterBuilder.from_json(
            Path(group_path, ".").glob("**/*.json"), profile_path` in params.py.
- ClusterBuilder is in a separate file, but I think I don't care how it works, as long as RESOURCE_GROUPS and RESOURCE_GROUP_PROFILES are set in my settings file, and the clusters are actually used for the custom generator (so I need my settings file to put the custom generator in a resource group).
- My settings file has `RESOURCE_GROUP_PROFILES: ./data/resource_profiles` which contains mostly parquet files with the actual profiles but also two csvs with site mapping.
- So, do I need to find the desired format of these parquet files and make one for wave data to put in resource_profiles folder? Or, do I want to add a profile column to the extra generators? The latter would be easier but I'm not sure yet if it works.

Look at `generators.py adjust_min_power_based_on_profile()`

In [11]:

settings = load_settings(settings_path)
settings["input_folder"] = settings_path.parent / settings["input_folder"]
scenario_definitions = pd.read_csv(
    settings["input_folder"] / settings["scenario_definitions_fn"]
)
scenario_settings = build_scenario_settings(settings, scenario_definitions)

pudl_engine, pudl_out, pg_engine = init_pudl_connection(
    freq="AS",
    start_year=min(settings.get("eia_data_years")),
    end_year=max(settings.get("eia_data_years")),
    pudl_db="sqlite:////home/becca/Documents/git/WEC-DECIDER/modules/CEM/data/pudl/pudl.sqlite", #os.fspath("sqlite:///" / cwd / "./data/pudl/pudl.sqlite"),
    pg_db="sqlite:////home/becca/Documents/git/WEC-DECIDER/modules/CEM/data/pg/pg_misc_tables_efs_2023.2.sqlite"#os.fspath("sqlite:///" / cwd / "./data/pg/pg_misc_tables_efs_2023.2.sqlite")
)

check_settings(settings, pg_engine)

gc = GeneratorClusters(pudl_engine, pudl_out, pg_engine, scenario_settings[2030]["Case_1"])

df = pd.DataFrame([g.group for g in gc.cluster_builder.groups])
print(df[['technology', 'profiles']])


Using PUDL database: sqlite:////home/becca/Documents/git/WEC-DECIDER/modules/CEM/data/pudl/pudl.sqlite



    *****************************
    The ATB technology "LandbasedWind_LTRG10" listed in your settings file under 'atb_new_gen'
    is not fully specified in the 'cost_multiplier_technology_map' settings parameter.
    Part of the <tech>_<tech_detail> string might be included, but it is best practice to
    include the full name in this format. Check your settings file.
        

    *****************************
    The ATB technology "OffShoreWind_OTRG10" listed in your settings file under 'atb_new_gen'
    is not fully specified in the 'cost_multiplier_technology_map' settings parameter.
    Part of the <tech>_<tech_detail> string might be included, but it is best practice to
    include the full name in this format. Check your settings file.
        

    *****************************
    The ATB technology "UtilityPV_Chicago" listed in your settings file under 'atb_new_gen'
    is not fully specified in the 'cost_multiplier_technology_map' settings parameter.
    Part of the <te

      technology                                           profiles
0      utilitypv  /home/becca/Documents/git/WEC-DECIDER/modules/...
1   offshorewind  /home/becca/Documents/git/WEC-DECIDER/modules/...
2   offshorewind  /home/becca/Documents/git/WEC-DECIDER/modules/...
3  landbasedwind  /home/becca/Documents/git/WEC-DECIDER/modules/...
4   offshorewind  /home/becca/Documents/git/WEC-DECIDER/modules/...
5   offshorewind  /home/becca/Documents/git/WEC-DECIDER/modules/...
6  landbasedwind  /home/becca/Documents/git/WEC-DECIDER/modules/...
7      utilitypv  /home/becca/Documents/git/WEC-DECIDER/modules/...
8          hydro  /home/becca/Documents/git/WEC-DECIDER/modules/...
9          hydro  /home/becca/Documents/git/WEC-DECIDER/modules/...


In [23]:

new_gen = gc.create_new_generators()
print(new_gen[['technology','profile','cap_recovery_years']])

Site assignment empty for renewables cluster in region NENG_CT for technology OffShoreWind_OTRG10_Mid with profile /home/becca/Documents/git/WEC-DECIDER/modules/CEM/data/resource_profiles/interpolated_scaled_offshore_wind_profiles_20210713.parquet and site map None.
Problem with renewables clustering for region NENG_CT. Expected to create 3 clusters, but actually made 2. Check that RESOURCE_GROUPS aligns with your region aggregations.
Site assignment empty for renewables cluster in region NENG_ME for technology OffShoreWind_OTRG10_Mid with profile /home/becca/Documents/git/WEC-DECIDER/modules/CEM/data/resource_profiles/interpolated_scaled_offshore_wind_profiles_20210713.parquet and site map None.
Problem with renewables clustering for region NENG_ME. Expected to create 3 clusters, but actually made 2. Check that RESOURCE_GROUPS aligns with your region aggregations.
/home/becca/Documents/git/PowerGenome/powergenome/external_data.py:142: FutureWarning: The frame.append method is deprecat

                   technology                                            profile  cap_recovery_years
0   NaturalGas_CCCCSAvgCF_Low                                                  0                  20
1      NaturalGas_CCAvgCF_Mid                                                  0                  15
2      NaturalGas_CTAvgCF_Mid                                                  0                  15
3               Battery_*_Mid                                                  0                  15
4                  Wave_500__  [0.659, 0.6692, 0.6794, 0.6896, 0.6886, 0.6876...                  20
..                        ...                                                ...                 ...
58             res_water_heat  [0.31627080721359263, 0.19286241595930836, 0.0...                <NA>
59             res_water_heat  [0.29648009817517074, 0.18278963623517078, 0.0...                <NA>
60           trans_light_duty  [0.11231639233145782, 0.043793405677439064, 0....           

In [7]:
all_gen=gc.create_all_generators()# new gen will have a "profile" column containing a list or np array

2025-07-11 13:50:35 [    INFO] catalystcoop.pudl.transform.eia861:456 Started with 325 missing BA Codes out of 12670 records (2.57%)
2025-07-11 13:50:37 [    INFO] catalystcoop.pudl.transform.eia861:480 Ended with 325 missing BA Codes out of 12670 records (2.57%)
2025-07-11 13:50:37 [    INFO] catalystcoop.pudl.output.eia860:177 97.4% of plant records have consistently reported BA Codes
2025-07-11 13:50:37 [    INFO] catalystcoop.pudl.output.eia860:227 Before any filling treatment has been applied. 2.6% of records have no BA codes
2025-07-11 13:50:37 [    INFO] catalystcoop.pudl.output.eia860:227 Backfilling and consistent value is the same. Filled w/ most consistent BA code. 2.6% of records have no BA codes
2025-07-11 13:50:37 [    INFO] catalystcoop.pudl.output.eia860:227 SWPP is most consistent value. Filled w/ oldest BA code. 2.6% of records have no BA codes
2025-07-11 13:50:37 [    INFO] catalystcoop.pudl.output.eia860:227 NWMT is most consistent value. Filled w/ oldest BA code. 2

In [8]:
%debug

> /tmp/ipykernel_2542064/813363919.py(5)<module>()
      2 pg_script = "./get-powergenome-data.sh"
      3 pg_output = "./results"
      4 
----> 5 subprocess.run(["bash",pg_script]) # run power genome to generate all template folders
      6 subprocess.run(["bash","./run_powergenome.sh"]) # run power genome to generate all template folders and run the analysis

--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user


In [ ]:
NREL_ATB_TECHNOLOGY_MAP = {
    ("utilitypv", None): {"technology": "utilitypv"},
    ("landbasedwind", None): {"technology": "landbasedwind"},
    ("offshorewind", None): {"technology": "offshorewind"},
    ("hydropower", None): {"technology": "hydro"},
    **{
        ("offshorewind", f"otrg{x}"): {
            "technology": "offshorewind",
            "turbine_type": "fixed",
        }
        for x in range(1, 8)
    },
    **{
        ("offshorewind", f"class{x}"): {
            "technology": "offshorewind",
            "turbine_type": "fixed",
        }
        for x in range(1, 8)
    },
    **{
        ("offshorewind", f"otrg{x}"): {
            "technology": "offshorewind",
            "turbine_type": "floating",
        }
        for x in range(8, 16)
    },
    **{
        ("offshorewind", f"class{x}"): {
            "technology": "offshorewind",
            "turbine_type": "floating",
        }
        for x in range(8, 16)
    },
}

for key, value in NREL_ATB_TECHNOLOGY_MAP.items():
    print(f"{key}: {value}")

clusters = copy.deepcopy(settings).get("renewables_clusters", [])

for scenario in clusters or []:
    technologies = [
            k
            for k, v in NREL_ATB_TECHNOLOGY_MAP.items()
            if v and all([scenario.get(ki) == vi for ki, vi in v.items()])
        ]
    print(f"Region: {scenario['region']}")
    print(f"Technologies: {technologies}")
    if not technologies:
        s = (
            f"You have a renewables_cluster for technology '{scenario.get('technology')} "
            f"in region '{scenario.get('region')}', but no comparable new-build technology "
            "was specified in your settings file."
        )
        print(s)

('utilitypv', None): {'technology': 'utilitypv'}
('landbasedwind', None): {'technology': 'landbasedwind'}
('offshorewind', None): {'technology': 'offshorewind'}
('hydropower', None): {'technology': 'hydro'}
('offshorewind', 'otrg1'): {'technology': 'offshorewind', 'turbine_type': 'fixed'}
('offshorewind', 'otrg2'): {'technology': 'offshorewind', 'turbine_type': 'fixed'}
('offshorewind', 'otrg3'): {'technology': 'offshorewind', 'turbine_type': 'fixed'}
('offshorewind', 'otrg4'): {'technology': 'offshorewind', 'turbine_type': 'fixed'}
('offshorewind', 'otrg5'): {'technology': 'offshorewind', 'turbine_type': 'fixed'}
('offshorewind', 'otrg6'): {'technology': 'offshorewind', 'turbine_type': 'fixed'}
('offshorewind', 'otrg7'): {'technology': 'offshorewind', 'turbine_type': 'fixed'}
('offshorewind', 'class1'): {'technology': 'offshorewind', 'turbine_type': 'fixed'}
('offshorewind', 'class2'): {'technology': 'offshorewind', 'turbine_type': 'fixed'}
('offshorewind', 'class3'): {'technology': '

In [ ]:
print(clusters == gc)
print(clusters == gc.cluster_list)
print(clusters == gc.cluster_builder.groups)

print('\n')
print(type(clusters))
print(type(gc))
print(type(gc.cluster_list))
print(type(gc.cluster_builder.groups))

print('\n')
print(type(clusters[0]))
print(type(gc.cluster_list[0]))
print(type(gc.cluster_builder.groups[0]))

print('\n')
print(clusters[0])
print(gc.cluster_list[0].columns)
print(type(gc.cluster_builder.groups[0].group))

print('\n')
print(gc.cluster_builder.groups[0].group)


# clusters (settings.get renewables clusters) is a list of dicts, with keys 'region', 'filter', and 'bin'
# gc.cluster_list is list of dataframes with columns 'cluster', 'technology', 'region', and specifics like id, heat rates, costs, capacities
# gc.cluster_builder.groups is list of ResourceGroup objects
# Each ResourceGroup object has a 'group' attribute that is a dict with keys 'existing', 'tree', 'technology', 'metadata', 'profiles', and 'site_map'

False
False
False


<class 'list'>
<class 'powergenome.generators.GeneratorClusters'>
<class 'list'>
<class 'list'>


<class 'dict'>
<class 'pandas.core.frame.DataFrame'>
<class 'powergenome.resource_clusters.ResourceGroup'>


{'region': 'NENGREST', 'technology': 'landbasedwind', 'filter': [{'feature': 'lcoe', 'max': 75}], 'bin': [{'feature': 'lcoe', 'weights': 'mw', 'q': 3}, {'feature': 'cf', 'q': 2}]}
Index(['cluster', 'winter_capacity_mw', 'capacity_mwh', 'minimum_load_mw', 'heat_rate_mmbtu_mwh', 'Fixed_OM_Cost_per_MWyr', 'Var_OM_Cost_per_MWh', 'heat_rate_mmbtu_mwh_iqr', 'heat_rate_mmbtu_mwh_std', 'fixed_o_m_mw_std', 'Min_Power', 'num_units', 'technology', 'region', 'plant_id_eia', 'unit_id_pg'], dtype='object')
<class 'dict'>


{'existing': False, 'tree': None, 'technology': 'utilitypv', 'metadata': PosixPath('data/resource_groups/eastern-20-zone/solar_lcoe_eastern_interconnect.csv'), 'profiles': PosixPath('data/resource_profiles/vce_site_solar_profiles_irl_1.28_20220913.parquet'),

In [ ]:

# this extracts the profile column and puts that data into a separate df
df = make_generator_variability(new_gen)

df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.317678,0.316271,0.296480,0.112316,0.104095,0.126692
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.194218,0.192862,0.182790,0.043793,0.040479,0.049809
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.081956,0.080843,0.077978,0.035463,0.032561,0.040726
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.090236,0.089008,0.087729,0.024303,0.022321,0.028570
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.065968,0.064967,0.065504,0.004474,0.004105,0.005398


In [29]:
# paths
pg_script = "./get-powergenome-data.sh"
pg_output = "./results"

subprocess.run(["bash",pg_script]) # run power genome to generate all template folders
subprocess.run(["bash","./run_powergenome.sh"]) # run power genome to generate all template folders and run the analysis

CompletedProcess(args=['bash', './run_powergenome.sh'], returncode=0)

File ‘./data_east/misc_gen_inputs.csv’ already there; not retrieving.

File ‘./data_east/demand_segments_voll.csv’ already there; not retrieving.

File ‘./data_east/resource_capacity_spur.csv’ already there; not retrieving.

File ‘./data_east/emission_policies.csv’ already there; not retrieving.

File ‘./data_east/Reserves.csv’ already there; not retrieving.

File ‘./data_CA/test_demand_segments_voll.csv’ already there; not retrieving.

File ‘./data_CA/test_misc_gen_inputs.csv’ already there; not retrieving.

/home/becca/miniconda3/envs/wec-decider-7-pg-edit/lib/python3.10/site-packages/pudl/__init__.py:3: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
14:47:12 [    INFO] powergenome:180 Reading settings file
14:47:13 [ WARNING] powergenome.util:445 
    *****************

Using data folder: /home/becca/Documents/git/WEC-DECIDER/modules/CEM/data
Using PUDL database: sqlite:////home/becca/Documents/git/WEC-DECIDER/modules/CEM/data/pudl/pudl.sqlite
PowerGenome ran for data_east.


In [ ]:
import sys
from powergenome.run_powergenome_multiple_outputs_cli import main as run_pg_cli
# alternate method to run CLI: unlike subprocess, this lets you use the python debugger on errors in PowerGenome

# Save the original sys.argv
original_argv = sys.argv.copy()

# Set sys.argv to what the script expects
sys.argv = [
    'run_powergenome_multiple_outputs_cli.py',  # script name
    '--settings_file', './data_east/settings',
    '--results_folder', './data_east/cases'
]

try:
    # Call the main function - it will parse the modified sys.argv
    run_pg_cli()
finally:
    # Restore the original sys.argv
    sys.argv = original_argv

18:29:40 [    INFO] powergenome:180 Reading settings file
18:29:40 [    INFO] powergenome:180 Reading settings file
18:29:40 [    INFO] powergenome:180 Reading settings file
18:29:40 [    INFO] powergenome:180 Reading settings file
18:29:40 [ WARNING] powergenome.util:445 
    *****************************
    The ATB technology "LandbasedWind_LTRG10" listed in your settings file under 'atb_new_gen'
    is not fully specified in the 'cost_multiplier_technology_map' settings parameter.
    Part of the <tech>_<tech_detail> string might be included, but it is best practice to
    include the full name in this format. Check your settings file.
        
18:29:40 [ WARNING] powergenome.util:445 
    *****************************
    The ATB technology "LandbasedWind_LTRG10" listed in your settings file under 'atb_new_gen'
    is not fully specified in the 'cost_multiplier_technology_map' settings parameter.
    Part of the <tech>_<tech_detail> string might be included, but it is best practic

In [32]:
%tb

SystemExit: 2

In [ ]:
import sqlalchemy as sa
pg_db = 'sqlite:////home/becca/Documents/git/WEC-DECIDER/modules/CEM/data/pg/_pg_misc_tables.sqlite3'
#pg_db = 'sqlite:///data/pg/_pg_misc_tables.sqlite3'
pg_engine = sa.create_engine(pg_db, pool_pre_ping=True)
c = pg_engine.connect()

In [ ]:

!pwd
# cd to WEC-DECIDER
#os.chdir('./../..')
os.chdir('modules/CEM')
!pwd


/home/becca/Documents/git/WEC-DECIDER/modules/CEM


FileNotFoundError: [Errno 2] No such file or directory: 'modules/CEM'

In [ ]:

# locations
# locs =["CAISO"]
locs =["CAISO"]
pg_id = 0

for loc in locs:
    template_folders = Path(pg_output).glob('**/'+loc)
    #print(list(template_folders))
    for template_folder in template_folders:
        print(template_folder)
        # jl.seval("""cd("modules/CEM")""")
        jl.seval("""Pkg.instantiate()""")
        jl.seval("""Pkg.status()""")
        jl.seval("""Pkg.project()""")
        
        jl.seval("""include("case_runner_new.jl")""")
        jl.seval("""run_caserunner(" """ + str(pg_id) + """ ", " """ + loc + """ ")""")
        pg_id += 1
        # run caserunner with the replacement.csv which corresponds to the proper location.
    

In [ ]:

print("Template folders found for", loc, ":", list(template_folders))

Template folders found for CAISO : []


The script below is running 3rd party caserunner. To run this file, uncomment 
include("/Users/kuankhaixin/WEC-DECIDER/modules/CEM/caserunner_v1.jl") in Run.jl. 

In [ ]:

# Determine the root directory of the project 
if "__file__" in globals():
    # Script context
    script_dir = os.path.dirname(os.path.abspath(__file__))
else:
    script_dir = os.getcwd()


root_dir = os.path.abspath(os.path.join(script_dir, "..", "..")) 
cem_dir = os.path.join(root_dir, "modules", "CEM")
case_folder = "case_1_0"
run_file_path = os.path.join(cem_dir, "Cases", case_folder, "Run.jl")
julia_project_path = cem_dir  

# Check that Run.jl exists
if not os.path.exists(run_file_path):
    print(f" Error: Run.jl not found at {run_file_path}")
    sys.exit(1)

# Launch Julia
print(f" Running GenX for case: {case_folder}...")

process = subprocess.Popen(
    ["julia", f"--project={julia_project_path}", run_file_path],
    cwd=root_dir,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,  # Merge stderr into stdout
    text=True,
    bufsize=1  # Line-buffered
)

# Stream all output
for line in process.stdout:
    print(line, end="")

process.wait()



Running GenX for case: case_1_0...
  Activating project at `~/WEC-DECIDER/modules/CEM`
    Building Gurobi → `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/0af671809e0cde0131a2f8607397197fd3648084/build.log`
Case 1 now creating
Case 2 now creating
Case 3 now creating
Case 4 now creating
Case 5 now creating
Case 6 now creating
Case 7 now creating
Case 8 now creating
Case 9 now creating
Case 10 now creating
Case 11 now creating
Case 12 now creating
Case 13 now creating
Case 14 now creating
Case 15 now creating
Case 16 now creating
Case 17 now creating
Case 18 now creating
Case 19 now creating
Case 20 now creating
Case 21 now creating
Case 22 now creating
Case 23 now creating
Case 24 now creating
Case 25 now creating


0

The script below is running GenX original caserunner. To run this file, uncomment 
run_genx_case!(dirname(@__FILE__), Gurobi.Optimizer) in Run.jl. 


In [ ]:

# Determine root_dir
if "__file__" in globals():
    # Running as a script
    script_dir = os.path.dirname(os.path.abspath(__file__))
else:
    script_dir = os.getcwd()

# Traverse up to WEC-DECIDER root
root_dir = os.path.abspath(os.path.join(script_dir, "..", ".."))  
cem_dir = os.path.join(root_dir, "modules", "CEM")
#run_file_path = os.path.join(cem_dir, "data_east", "results", case_folder, "Run.jl")
run_file_path = os.path.join(cem_dir, "Run.jl")
julia_project_path = cem_dir  

# Confirm the script exists
if not os.path.exists(run_file_path):
    print(f"Run.jl not found at {run_file_path}")
else:

    process = subprocess.Popen(
        ["julia", f"--project={julia_project_path}", run_file_path],
        cwd=root_dir,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )

    # Live stdout printing
    for line in process.stdout:
        print(line, end="")

    stdout, stderr = process.communicate()

    if stderr.strip():
        print("\n Julia stderr:")
        print(stderr)



Running GenX for case: Case_1 of 5
  ____           __  __   _ _
 / ___| ___ _ __ \ \/ /  (_) |
| |  _ / _ \ '_ \ \  /   | | |
| |_| |  __/ | | |/  \ _ | | |
 \____|\___|_| |_/_/\_(_)/ |_|
                       |__/
 Version: 0.4.4

Configuring Settings
Configuring Multistage Settings
Configuring Solver
Reading Input CSV Files
Network.csv Successfully Read!
Demand (load) data Successfully Read!
Fuels_data.csv Successfully Read!

Summary of resources loaded into the model:
-------------------------------------------------------
	Resource type 		Number of resources
	Thermal        		24
	VRE            		52
	Hydro          		3
	Storage        		8
	Must_run       		6
	Flexible_demand		6
Total number of resources: 99
-------------------------------------------------------
Generators_variability.csv Successfully Read!
Validating time basis
Capacity_reserve_margin.csv Successfully Read!
Energy_share_requirement.csv Successfully Read!
CSV Files Successfully Read In From /home/becca/Documents/